# Data quality
In this notebook we analyze the data quality refered to COVID19 in Mexico. This information in obtained thought the Secretariat of Health and published by *Dirección General de Epidemiología* the mexican CDC. https://www.gob.mx/salud/documentos/datos-abiertos-152127. 

In [1]:

import glob
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import datetime
from IPython.display import Markdown as md

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import json

%matplotlib 

Using matplotlib backend: MacOSX


In [2]:
dic_ENTIDADES = d2 = json.load(open("../src/diccionario_datos_covid19/ENTIDADES.jsn"))
dic_MUNICIPIOS = d2 = json.load(open("../src/diccionario_datos_covid19/MUNICIPIOS.jsn"))
dic_NACIONALIDAD = d2 = json.load(open("../src/diccionario_datos_covid19/NACIONALIDAD.jsn"))
dic_ORIGEN = d2 = json.load(open("../src/diccionario_datos_covid19/ORIGEN.jsn"))
dic_RESULTADO = d2 = json.load(open("../src/diccionario_datos_covid19/RESULTADO.jsn"))
dic_SECTOR = d2 = json.load(open("../src/diccionario_datos_covid19/SECTOR.jsn"))
dic_SEXO = d2 = json.load(open("../src/diccionario_datos_covid19/SEXO.jsn"))
dic_SI_NO = d2 = json.load(open("../src/diccionario_datos_covid19/SI_NO.jsn"))
dic_TIPO_PACIENTE = d2 = json.load(open("../src/diccionario_datos_covid19/TIPO_PACIENTE.jsn"))

## Load data

In [3]:

# load the csv from the src folder
strpath = '../src/'
m = []
i =0
if not os.path.isfile('fulldb1.csv'):
    for filename in glob.glob(strpath+"*"):
        if filename.find('.csv')>0:
            date = '20'+filename[7:9] +'-' + filename[11:13] +'-' +filename[13:15]
            #print(date)
            df = pd.read_csv(filename, encoding = "ISO-8859-1",dtype =str)
            df['FechaRegistro'] = date
            df['MUNICIPIO']= df['ENTIDAD_RES'] + df['MUNICIPIO_RES']
            df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)
            df.replace({'ORIGEN':dic_ORIGEN},inplace=True)
            df.replace({"SECTOR": dic_SECTOR},inplace=True)
            df.replace({"ENTIDAD_UM": dic_ENTIDADES},inplace=True)
            df.replace({"SEXO": dic_SEXO},inplace=True)
            df.replace({"ENTIDAD_NAC": dic_ENTIDADES},inplace=True)
            df.replace({"ENTIDAD_RES": dic_ENTIDADES},inplace=True)
            df.replace({"MUNICIPIO": dic_MUNICIPIOS},inplace=True)
            df['MUNICIPIO_RES']= df['MUNICIPIO']
            df.replace({"TIPO_PACIENTE": dic_TIPO_PACIENTE},inplace=True)
            df.replace({'INTUBADO': dic_SI_NO},inplace=True)
            df.replace({'NEUMONIA': dic_SI_NO},inplace=True)
            df.replace({'NACIONALIDAD': dic_NACIONALIDAD},inplace=True)
            df.replace({'EMBARAZO': dic_SI_NO},inplace=True)
            df.replace({'HABLA_LENGUA_INDIG': dic_SI_NO},inplace=True)
            df.replace({'DIABETES': dic_SI_NO},inplace=True)
            df.replace({'EPOC': dic_SI_NO},inplace=True)
            df.replace({'ASMA': dic_SI_NO},inplace=True)
            df.replace({'INMUSUPR': dic_SI_NO},inplace=True)
            df.replace({'HIPERTENSION': dic_SI_NO},inplace=True)
            df.replace({'OTRA_COM': dic_SI_NO},inplace=True)
            df.replace({'CARDIOVASCULAR': dic_SI_NO},inplace=True)
            df.replace({'OBESIDAD': dic_SI_NO},inplace=True)
            df.replace({'RENAL_CRONICA': dic_SI_NO},inplace=True)
            df.replace({'TABAQUISMO': dic_SI_NO},inplace=True)
            df.replace({'OTRO_CASO': dic_SI_NO},inplace=True)
            df.replace({'RESULTADO': dic_RESULTADO},inplace=True)
            df.replace({'MIGRANTE': dic_SI_NO},inplace=True)
            df.replace({'UCI': dic_SI_NO},inplace=True)
            if i== 0:
                df0=df.copy()
                i=1
            else:
                frame = [df0,df]
                df0 = pd.concat(frame)
            del df
else:
    df0 = pd.read_csv('fulldb.csv')
    df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')
    lastdateRecord = df0.FechaRegistro.max()
    deltaT = datetime.datetime.now() - lastdateRecord
    for i in range(1,deltaT.days+1) :
        filename = (lastdateRecord + datetime.timedelta(days=1)).strftime("%Y%m%d")+'.csv'
        df = pd.read_csv(strpath+filename, encoding = "ISO-8859-1",dtype =str)
        df['FechaRegistro'] = (lastdateRecord + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)

len(df0)

32543335

In [4]:
df0['LastDay']  = df0.groupby('ID_REGISTRO')['FechaRegistro'].transform('max')
df0['FirstDay'] = df0.groupby('ID_REGISTRO')['FechaRegistro'].transform('min')
df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')
df0['Duration'] = (df0.LastDay - df0.FirstDay).dt.days
df0['numRecords']  = df0.groupby('ID_REGISTRO')['FechaRegistro'].transform('count')
# df0['numRecords']  = df0.groupby('ID_REGISTRO')['FechaRegistro'].transform('min')
df0['DurationRecords'] = df0['Duration'] - df0['numRecords']
df0 = df0.sort_values(by='FechaRegistro', ascending=False)
df0.head()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
df0['Duration'] = (df0.LastDay - df0.FirstDay).dt.days
df0['numRecords']  = df0.groupby('ID_REGISTRO')['FechaRegistro'].transform('count')
# df0['numRecords']  = df0.groupby('ID_REGISTRO')['FechaRegistro'].transform('min')
df0['DurationRecords'] = df0['Duration'] - df0['numRecords']
df0 = df0.sort_values(by='FechaRegistro', ascending=False)
df0.head()

In [ ]:
md("We have {0} records of covid evolution cases in Mexico until today {1}".format(df0.shape[0], datetime.datetime.now()) )


In [ ]:
1

In [ ]:
#today = (datetime.datetime.now()).strftime("%Y%m%d")
today = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y%m%d")
yesterday = (datetime.datetime.now() - datetime.timedelta(days=2)).strftime("%Y%m%d")
ereyesterday = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime("%Y%m%d")
df= pd.read_csv( '../src/'+today       +'.csv', encoding = "ISO-8859-1",dtype =str)
dfA= pd.read_csv('../src/'+yesterday   +'.csv', encoding = "ISO-8859-1",dtype =str)
dfB= pd.read_csv('../src/'+ereyesterday+'.csv', encoding = "ISO-8859-1",dtype =str)

## 1 How many cases we have in the last report vs db?

In [ ]:
# we get unique cases globaly and from the last csv 
global_ids = df0.ID_REGISTRO.unique().tolist()
last_ids  = df.ID_REGISTRO.unique().tolist()
# we get the cases that are present in previous reports but they doesn't show in the last report
untracket_cases = set(global_ids)- set(last_ids)
tracket_cases = set(last_ids)

In [ ]:
df0.FECHA_SINTOMAS = pd.to_datetime(df0['FECHA_SINTOMAS'], format ='%Y-%m-%d')

In [ ]:
# we extract the records that does not exist in the last report
df1 = df0[df0.ID_REGISTRO.isin(untracket_cases)]
df1 = df1.copy()

md("we have {0} register that are not tracket".format(df1.ID_REGISTRO.nunique()) )

1. How many days the records exist
2. Is there some kind or relation between the records?

In [ ]:
df1.FECHA_SINTOMAS = pd.to_datetime(df1['FECHA_SINTOMAS'], format ='%Y-%m-%d')
df1.FECHA_ACTUALIZACION = pd.to_datetime(df1['FECHA_ACTUALIZACION'], format ='%Y-%m-%d')
df1.FechaRegistro = pd.to_datetime(df1['FechaRegistro'], format ='%Y-%m-%d')


In [ ]:
# we get the duration of cases between records
df1['LastDay']  = df1.groupby('ID_REGISTRO')['FechaRegistro'].transform('max')
df1['FirstDay'] = df1.groupby('ID_REGISTRO')['FechaRegistro'].transform('min')
df1['Duration'] = (df1.LastDay - df1.FirstDay).dt.days

In [ ]:
# We keep only one case to get plot correctly the cases
df3 = df1.drop_duplicates(keep='first')
df3 = df3.drop_duplicates('ID_REGISTRO',keep='first')
df_3 = df3.groupby(['RESULTADO','Duration'])['Duration'].count().to_frame(name = 'count').reset_index()
# sns.lineplot(x='Duration', y='count', data=df_3, hue='RESULTADO');

In [ ]:
len(df1[df1.Duration!=0].drop_duplicates(['ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI']))

In [ ]:
len(df1)

In [ ]:
len(df1[df1.Duration!=0])

In [ ]:
df1.ID_REGISTRO.nunique()

In [ ]:
df1R = df1[df1.Duration!=0].drop_duplicates(['ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'])

In [ ]:
df1R.head()

In [ ]:
df1[df1.Duration==0].drop_duplicates(['ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI']).head()

In [ ]:
df1[df1.ID_REGISTRO=='1288df']

In [ ]:
df3.shape[0]-df3[df3.Duration==0].shape[0]

In [ ]:
df_3.head(30)

In [ ]:
df_3 = df3.groupby(['Duration'])['Duration'].count().to_frame(name = 'count').reset_index()
sns.lineplot(x='Duration',y='count',  data=df_3[['Duration','count']],)#, palette=sns.color_palette("Set1", 3))

In [ ]:
sns.lineplot(x='Duration', y='count', data=df_3, style='RESULTADO')#, palette=sns.color_palette("Set1", 3))

In [ ]:
df3['EDAD_int'] = df3.EDAD.astype(int)

In [ ]:
df3.groupby(['RESULTADO','Duration'])['Duration'].count().sort_values(by='FechaRegistro', ascending=False)#.to_frame(name = 'count').reset_index()

In [ ]:
df3.groupby('EDAD_int')['EDAD_int'].count().plot()

In [ ]:
df1.FechaRegistro = pd.to_datetime(df1['FechaRegistro'], format='%Y-%m-%d')
df1 = df1.sort_values(by='FechaRegistro', ascending=False)

In [ ]:
df3.shape

In [ ]:
print(len(df1))
df3 = df1.drop_duplicates(keep='first')
# print(len(df3))
df3 = df3.drop_duplicates('ID_REGISTRO',keep='first')
# print(len(df3))
print(len(df1))

In [ ]:
df3.columns

In [ ]:
df3.groupby(['ORIGEN'])['ORIGEN'].count().sort_values(ascending=False).head(19)

In [ ]:
df3.groupby('SECTOR')['SECTOR'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('ENTIDAD_UM')['ENTIDAD_UM'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('SEXO')['SEXO'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('ENTIDAD_NAC')['ENTIDAD_NAC'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('ENTIDAD_NAC')['ENTIDAD_NAC'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('ENTIDAD_RES')['ENTIDAD_RES'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('MUNICIPIO_RES')['MUNICIPIO_RES'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('TIPO_PACIENTE')['TIPO_PACIENTE'].count()

In [ ]:
df3.groupby('INTUBADO')['INTUBADO'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('NEUMONIA')['NEUMONIA'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('EDAD')['EDAD'].count().sort_values( ascending=False)

In [ ]:
df3['EDAD_int'] = df3.EDAD.astype(int)
df3.groupby('EDAD_int')['EDAD_int'].count().plot()

In [ ]:
df3.shape

In [ ]:
df3.groupby('EMBARAZO')['EMBARAZO'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('HABLA_LENGUA_INDIG')['HABLA_LENGUA_INDIG'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('DIABETES')['DIABETES'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('ASMA')['ASMA'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('INMUSUPR')['INMUSUPR'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('HIPERTENSION')['HIPERTENSION'].count().sort_values( ascending=False)

In [ ]:
df3.columns

In [ ]:
df3.groupby('OTRA_COM')['OTRA_COM'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('CARDIOVASCULAR')['CARDIOVASCULAR'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('OBESIDAD')['OBESIDAD'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('RENAL_CRONICA')['RENAL_CRONICA'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('TABAQUISMO')['TABAQUISMO'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('OTRO_CASO')['OTRO_CASO'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('RESULTADO')['RESULTADO'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('MIGRANTE')['MIGRANTE'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('PAIS_NACIONALIDAD')['PAIS_NACIONALIDAD'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('PAIS_ORIGEN')['PAIS_ORIGEN'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('PAIS_ORIGEN')['PAIS_ORIGEN'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('UCI')['UCI'].count().sort_values( ascending=False)

In [ ]:
df3.groupby('EDAD')['EDAD'].count().sort_values( ascending=False)

# personas que siguen en el sistema

In [ ]:
# we extract the records that does not live in the last report
dfO1 = df0[df0.ID_REGISTRO.isin(tracket_cases)]
dfO1 = dfO1.copy()
dfO1.shape


In [ ]:
dfO1.columns

In [ ]:
# df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')
dfO1.FECHA_SINTOMAS = pd.to_datetime(dfO1['FECHA_SINTOMAS'], format ='%Y-%m-%d')
dfO1.FECHA_ACTUALIZACION = pd.to_datetime(dfO1['FECHA_ACTUALIZACION'], format ='%Y-%m-%d')
dfO1.FechaRegistro = pd.to_datetime(dfO1['FechaRegistro'], format ='%Y-%m-%d')
dfO1['delta_lastRecord'] = (dfO1.FECHA_ACTUALIZACION-dfO1.FechaRegistro ).dt.days
dfO1['deltaReporte'] = (dfO1.FechaRegistro-dfO1.FECHA_SINTOMAS).dt.days
dfO1[dfO1.FECHA_SINTOMAS<'2020-02-29'].groupby('RESULTADO')['RESULTADO'].count()



In [ ]:
# sort_values(by='FechaRegistro', ascending=False).head()

dfO13 = dfO1.drop_duplicates(['ID_REGISTRO', 
       'FECHA_INGRESO', 'FECHA_SINTOMAS', ],keep='first')
dfO13 = dfO13.copy()

In [ ]:
dfO13.groupby('ID_REGISTRO')['FECHA_ACTUALIZACION'].count().sort_values( ascending=False)

In [ ]:
dfO131 = dfO13[dfO13.FECHA_SINTOMAS<'2020-02-29']
dfO131.groupby('ID_REGISTRO')['FECHA_ACTUALIZACION'].count().sort_values( ascending=False).head(10)

In [ ]:
dfO131[dfO131.ID_REGISTRO.isin(['0304ae','02a0df','0226ad','13fb66','0632c1','1732a3',])][['ID_REGISTRO', 
        'FECHA_SINTOMAS','FECHA_INGRESO',]].sort_values(['ID_REGISTRO','FECHA_SINTOMAS'], ascending=False)

In [ ]:
dfO131['deltaCorrección'] = (dfO131.FECHA_INGRESO-dfO131.FECHA_SINTOMAS).dt.days

In [ ]:
df0['FECHA_SINTOMAS'].min()

In [ ]:
df0[df0['FECHA_SINTOMAS']=='2020-01-01']

In [ ]:
first= pd.read_csv( '../src/'+'20200412'       +'.csv', encoding = "ISO-8859-1",dtype =str)


In [ ]:
first.FECHA_SINTOMAS = pd.to_datetime(first['FECHA_SINTOMAS'], format ='%Y-%m-%d')

In [ ]:
first.FECHA_SINTOMAS.min()

At the end of 2019, the world saw the born of the Sars-cov-2 virus which genereate a disease called COVID19 (COrona VIrus disease). The high risk that we face with this virus is that the sympthons are related to a typical flu but generate a high damage un the lungs decreasing the oxygen level. The disminution of oxygen generate a high demand of intensive beds.

From WHO files

According to the WHO the incubation period for COVID-19, is on average 5-6 days, but can be extended to 14 days. However there some evidence that shows that exist different types of transmition, such as symptomatic, pre-symptomatic and a symptomatic.

The typical transmition is the symptomatic, this ocurrs when a person shown COVID symptomatic and can spread the virus conciently or inconciently respiratory droplest over thier enviroment. However, some cases has shown that a person can transmit COVID-19 without symptoms, (3 or 2 days) before present the typical symptoms. Finally there are some concern about the asyptomatic transmission according to the WHO: Asymptomatic transmission refers to transmission of the virus from a person, who does not develop symptoms. There are few reports of laboratory-confirmed cases who are truly asymptomatic, and to date, there has been no documente asymptomatic transmission. This does not exclude the possibility that it may occur. Asymptomatic cases have been reported as part of contact tracing efforts in some countries. Source https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200402-sitrep-73-covid-19.pdf?sfvrsn=5ae25bc7_2#:~:text=An%20asymptomatic%20laboratory%2Dconfirmed,more%20information%20becomes%20available.


The goal of this notebook is listed below:
* Understand the evolution of the covi19 in Mexico, there are a rate of 10% of deads though a visualization analysis


Key elements

* Incubation period can be expresed a GAMMA.
* Is most common the transmision after show symptoms.
* How we can modeled (explain pre-symtomatic transmition)
* how we can modeled asyntomatic transmition


In [ ]:
df.head()

In [ ]:
df

In [ ]:
dic_ORIGEN

In [ ]:
for each in dic_ORIGEN:
    dic_ORIGEN[each]= dic_ORIGEN[each][0]

dic_ORIGEN
dic_NACIONALIDAD
dic_SEXO
dic_SI_NO
dic_TIPO_PACIENTE
dic_SECTOR
dic_RESULTADO

In [ ]:
dic_NACIONALIDAD
dic_ORIGEN
dic_RESULTADO
dic_SECTOR
dic_SEXO
dic_SI_NO
dic_TIPO_PACIENTE

In [ ]:
dic_ENTIDADES = d2 = json.load(open("../src/diccionario_datos_covid19/ENTIDADES.jsn"))
dic_MUNICIPIOS = d2 = json.load(open("../src/diccionario_datos_covid19/MUNICIPIOS.jsn"))
dic_NACIONALIDAD = d2 = json.load(open("../src/diccionario_datos_covid19/NACIONALIDAD.jsn"))
dic_ORIGEN = d2 = json.load(open("../src/diccionario_datos_covid19/ORIGEN.jsn"))
dic_RESULTADO = d2 = json.load(open("../src/diccionario_datos_covid19/RESULTADO.jsn"))
dic_SECTOR = d2 = json.load(open("../src/diccionario_datos_covid19/SECTOR.jsn"))
dic_SEXO = d2 = json.load(open("../src/diccionario_datos_covid19/SEXO.jsn"))
dic_SI_NO = d2 = json.load(open("../src/diccionario_datos_covid19/SI_NO.jsn"))
dic_TIPO_PACIENTE = d2 = json.load(open("../src/diccionario_datos_covid19/TIPO_PACIENTE.jsn"))